## Limpieza de datos para modelo de estimación de precio de vivienda

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.9f' % x)

### 1. Cargue de datos

In [3]:
df_train = pd.read_excel("../datos/entrenamiento_precios_vivienda.xlsx", dtype=str)

In [4]:
df_train['valor_total_avaluo'].describe()

count     12857
unique    12298
top           0
freq        160
Name: valor_total_avaluo, dtype: object

In [5]:
df_train['valor_total_avaluo'].isna().sum()

0

### 2. Eliminación de registros con datos errados

Se identifican algunos registros con datos errados luego de un campo con datos de desripción, esto seguramente se debe a caracteres especiales que hacen que se separe el campo de descripción y se desplacen datos hacia la derecha. Se eliminan estos registros dado que son pocos.

Para el campo acueducto_en_el_predio, sus valores normales son 0, Si o No. Se descartan registros cuyo valor sea diferente.

In [6]:
df_train_ok1 = df_train[df_train['acueducto_en_el_predio'].isin(["0","Si","No"])].copy()
df_train_bad1 = df_train[~df_train['acueducto_en_el_predio'].isin(["0","Si","No"])].copy()
df_train_bad1['campo'] = 'acueducto_en_el_predio'

In [7]:
df_train_ok1.shape

(12824, 221)

In [8]:
df_train_bad1.shape

(33, 222)

Para el campo condicion_ph, sus valores normales son 0, Construccion o Terreno. Se descartan registros cuyo valor sea diferente.

In [9]:
df_train_ok2 = df_train_ok1[df_train_ok1['condicion_ph'].isin(["0","Construccion","Terreno"])].copy()
df_train_bad2 = df_train_ok1[~df_train_ok1['condicion_ph'].isin(["0","Construccion","Terreno"])].copy()
df_train_bad2['campo'] = 'condicion_ph'

In [10]:
df_train_ok2.shape

(12735, 221)

In [11]:
df_train_bad2.shape

(89, 222)

El campo habitaciones debe ser numérico

In [12]:
df_train_ok3 = df_train_ok2[df_train_ok2['habitaciones'].str.isnumeric()].copy()
df_train_bad3 = df_train_ok2[~df_train_ok2['habitaciones'].str.isnumeric()].copy()
df_train_bad3['campo'] = 'habitaciones'

In [13]:
df_train_ok3.shape

(12728, 221)

In [14]:
df_train_bad3.shape

(7, 222)

Para el campo estado_acabados_pisos, sus valores normales son Bueno, Malo, Regular o Sin acabados. Se descartan registros cuyo valor sea diferente.

In [15]:
df_train_ok4 = df_train_ok3[df_train_ok3['estado_acabados_pisos'].isin(["Bueno","Malo","Regular","Sin acabados"])].copy()
df_train_bad4 = df_train_ok3[~df_train_ok3['estado_acabados_pisos'].isin(["Bueno","Malo","Regular","Sin acabados"])].copy()
df_train_bad4['campo'] = 'estado_acabados_pisos'

In [16]:
df_train_ok4.shape

(12715, 221)

In [17]:
df_train_bad4.shape

(13, 222)

Para el campo metodo_valuacion_1, sus valores normales son 0 y Comparación de  Mercado. Se descartan registros cuyo valor sea diferente.

In [18]:
df_train_ok5 = df_train_ok4[df_train_ok4['metodo_valuacion_1'].isin(["0","Comparación de  Mercado"])].copy()
df_train_bad5 = df_train_ok4[~df_train_ok4['metodo_valuacion_1'].isin(["0","Comparación de  Mercado"])].copy()
df_train_bad5['campo'] = 'metodo_valuacion_1'

In [19]:
df_train_ok5.shape

(12707, 221)

In [20]:
df_train_bad5.shape

(8, 222)

El campo valor_total_avaluo debe ser mayor a cero

In [21]:
df_train_ok6 = df_train_ok5[~df_train_ok5['valor_total_avaluo'].isin(["0"])].copy()
df_train_bad6 = df_train_ok5[df_train_ok5['valor_total_avaluo'].isin(["0"])].copy()
df_train_bad6['campo'] = 'valor_total_avaluo'

In [22]:
df_train_ok6.shape

(12697, 221)

In [23]:
df_train_bad6.shape

(10, 222)

Unificación de base con errores en datos

In [24]:
df_train_bad = pd.concat([df_train_bad1,df_train_bad2,df_train_bad3,df_train_bad4,df_train_bad5,df_train_bad6])
df_train_bad.shape

(160, 222)

In [25]:
df_train_bad['campo'].value_counts()

condicion_ph              89
acueducto_en_el_predio    33
estado_acabados_pisos     13
valor_total_avaluo        10
metodo_valuacion_1         8
habitaciones               7
Name: campo, dtype: int64

### 3. Tratamiento de campos de valor del avalúo

Realizamos la conversión de los campos de valor en valores flotantes, reemplazando las comas por puntos

In [26]:
def isfloat(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

In [27]:
df_train_ok6['clean_valor_total_avaluo'] = df_train_ok6['valor_total_avaluo'].apply(lambda v: float(v.replace(",",".")) if isfloat(v.replace(",",".")) else np.nan)
df_train_ok6['clean_valor_uvr'] = df_train_ok6['valor_uvr'].apply(lambda v: float(v.replace(",",".")) if isfloat(v.replace(",",".")) else np.nan)
df_train_ok6['clean_valor_avaluo_en_uvr'] = df_train_ok6['valor_avaluo_en_uvr'].apply(lambda v: float(v.replace(",",".")) if isfloat(v.replace(",",".")) else np.nan)

In [28]:
df_train_ok6['clean_valor_total_avaluo'].sort_values(ascending=True).head(10)

11233          3.000000000
6612         143.330000000
12722        300.000000000
2798    12000000.000000000
10821   13415745.000000000
882     15750000.000000000
8465    16500127.000000000
11511   17400000.000000000
1815    19911700.000000000
5353    20000000.000000000
Name: clean_valor_total_avaluo, dtype: float64

In [29]:
df_train_ok6['clean_valor_total_avaluo'].sort_values(ascending=False).head(10)

2011   5744320000008.000000000
316    4750621999950.000000000
2596   3530624600000.000000000
3292   1828219999961.000000000
5      1648560000516.000000000
909    1295616000946.000000000
90     1035720540032.000000000
1105    999196500000.000000000
155     908419998960.000000000
409     624999999429.000000000
Name: clean_valor_total_avaluo, dtype: float64

Se identifican algunos valores muy pequelos y otros demasiado grandes en el campo de valor del avalúo, para tener una validación si los valores son correctos o errados, los compararemos con el valor del avalúo en uvr.

In [30]:
df_train_ok6['clean_valor_uvr'].value_counts()

255.795200000                    74
254.182800000                    73
2518034.000000000                71
2521275000000000000.000000000    71
252104.000000000                 70
259.602100000                    70
259.672200000                    69
254.239800000                    68
253.840900000                    67
2516472000000000000.000000000    67
259.551900000                    66
2520467.000000000                66
2521073000000000000.000000000    66
2516513000000000000.000000000    66
254.296800000                    66
2520889.000000000                66
2521006000000000000.000000000    65
2520174.000000000                65
256.985900000                    65
257.335300000                    65
257.314700000                    65
2521191000000000000.000000000    64
2517682.000000000                64
259.514300000                    64
259.777900000                    64
256.204100000                    64
2519444.000000000                63
259.025600000               

El UVR es un valor de 3 cifras enteras con decimales, se observan valores con más de 3 cifras enteras, se asumirá que para estos valores se perdió el valor decimal y se realizará corrección de dichos valores.

In [31]:
df_train_ok6['clean_valor_uvr'] = df_train_ok6['clean_valor_uvr'].apply(lambda v: v/(10**(len(str(int(v)))-3)))

In [32]:
df_train_ok6['clean_valor_uvr'].describe()

count   12697.000000000
mean      255.783987548
std         3.136274532
min       251.639100000
25%       252.107300000
50%       256.379600000
75%       259.025600000
max       260.192300000
Name: clean_valor_uvr, dtype: float64

Se observa que luego de la limpieza anterior, los valores UVR tienen todos sentido.

Calculemos el valor en pesos teniendo en cuenta el valor del UVR, y verifiquemos la diferencia con respecto al valor del avalúo

In [33]:
df_train_ok6['val'] = df_train_ok6['clean_valor_uvr'] * df_train_ok6['clean_valor_avaluo_en_uvr']
df_train_ok6['diff'] = abs(df_train_ok6['clean_valor_total_avaluo'] - df_train_ok6['val'])

In [34]:
df_train_ok6.sort_values(by='clean_valor_avaluo_en_uvr',ascending=False)[['id','clean_valor_uvr','clean_valor_total_avaluo','clean_valor_avaluo_en_uvr', 'val', 'diff']].sort_values(by='diff',ascending=False).head(100)

,id,clean_valor_uvr,clean_valor_total_avaluo,clean_valor_avaluo_en_uvr,val,diff
2011,2883,252.004400000,5744320000008.000000000,2279452.260000000,574431999.109943986,5743745568008.889648438
316,449,251.667500000,4750621999950.000000000,1887627.370000000,475054461.139474988,4750146945488.860351562
2596,3742,252.053200000,3530624600000.000000000,14007457.949999999,3530624600.162940025,3527093975399.836914062
3292,4730,252.107300000,1828219999961.000000000,725175.350000000,182821999.515055001,1828037177961.484863281
5,7,251.696000000,1648560000516.000000000,654970.200000000,164853379.459199995,1648395147136.540771484
909,1319,251.709500000,1295616000946.000000000,514726.700000000,129561600.293650001,1295486439345.706298828
90,131,251.683800000,1035720540032.000000000,411516.570000000,103572054.100566000,1035616967977.899414062
1105,1599,251.768200000,999196500000.000000000,3968716.070000000,999196501.254974008,998197303498.744995117
155,221,251.679700000,908419998960.000000000,360942.900000000,90842000.789130002,908329156959.210815430
409,586,251.697800000,624999999429.000000000,248313.650000000,62499999.414969996,624937499429.585083008


In [35]:
df_train_ok6.query("diff > 1000000").shape

(52, 226)

Existen 52 registros con diferencias de más de un millón de pesos, al verificar los datos del valor del avalúo en pesos vs la converisón en UVR se puede identificar que parece que los errores son por mala localización de decimales o problemas de redondeos. Para remediarlo, para estos 52 registros dejaremos el valor del avalúo como la conversión a partir del UVR.

In [36]:
df_train_ok6[["clean_valor_total_avaluo",'diff']].sort_values("clean_valor_total_avaluo").head()

,clean_valor_total_avaluo,diff
11233,3.000000000,0.402095000
6612,143.330000000,0.373728000
12722,300.000000000,1.208500000
2798,12000000.000000000,0.973963000
10821,13415745.000000000,0.750120001


Así mismo, existen 3 registros con valores de avalúo muy pequeños y que concuerdan con el valor de UVR, se eliminarán estos registros ya que no hacen sentido su valor.

In [37]:
df_train_ok6['clean_valor_total_avaluo'] = df_train_ok6.apply(lambda r: r['val'] if r['diff'] > 1000000 else r['clean_valor_total_avaluo'], axis = 1)

In [38]:
df_train_clean = df_train_ok6.query("clean_valor_total_avaluo > 100000").copy()

In [39]:
df_train_clean.shape

(12694, 226)

In [40]:
df_train.shape[0] - df_train_clean.shape[0]

163

Luego de la limpieza se eliminan 163 registros

In [41]:
df_train_clean[['id','clean_valor_total_avaluo']].sort_values(by='clean_valor_total_avaluo',ascending=False).head(10)

,id,clean_valor_total_avaluo
10873,15534,14006855580.000000000
6632,9465,11847467000.000000000
7375,10532,8015111500.000000000
7334,10471,5121268000.000000000
12289,17531,3992887200.000000000
1731,2467,3965284042.000000000
5683,8089,3809417000.000000000
5845,8320,3533325000.000000000
2596,3742,3530624600.162940025
6486,9258,3201147000.000000000


In [42]:
df_train_clean.head()

,fecha_aprobación,objeto,motivo,proposito,tipo_avaluo,tipo_credito,tipo_subsidio,departamento_inmueble,municipio_inmueble,barrio,sector,direccion_inmueble_informe,alcantarillado_en_el_sector,acueducto_en_el_sector,gas_en_el_sector,energia_en_el_sector,telefono_en_el_sector,vias_pavimentadas,sardineles_en_las_vias,andenes_en_las_vias,estrato,barrio_legal,topografia_sector,condiciones_salubridad,transporte,demanda_interes,paradero,alumbrado,arborizacion,alamedas,ciclo_rutas,nivel_equipamiento_comercial,descripcion_general_sector,perspectivas_de_valorizacion,actualidad_edificadora,comportamiento_oferta_demanda,alcantarillado_en_el_predio,acueducto_en_el_predio,gas_en_el_predio,energia_en_el_predio,telefono_en_el_predio,tipo_inmueble,descripcion_tipo_inmueble,uso_actual,descripcion_uso_inmueble,clase_inmueble,descripcion_clase_inmueble,ocupante,sometido_a_propiedad_horizontal,altura_permitida,observaciones_altura_permitida,aislamiento_posterior,observaciones_aislamiento_posterior,aislamiento_lateral,observaciones_aislamiento_lateral,antejardin,observaciones_antejardin,indice_ocupacion,observaciones_indice_ocupacion,indice_construccion,observaciones_indice_construccion,predio_subdividido_fisicamente,unidades,contadores_agua,contadores_luz,accesorios,area_valorada,observaciones_generales_inmueble,condicion_ph,numero_piso,numero_de_edificios,area_actividad,uso_principal_ph,rph,porteria,citofono,bicicletero,piscina,tanque_de_agua,club_house,garaje_visitantes,teatrino,sauna,vigilancia_privada,tipo_vigilancia,administracion,vetustez,pisos_bodega,estructura,ajustes_sismoresistentes,cubierta,fachada,tipo_fachada,estructura_reforzada,danos_previos,material_de_construccion,detalle_material,iluminacion,ventilacion,irregularidad_planta,irregularidad_altura,observaciones_estructura,habitaciones,estar_habitacion,cuarto_servicio,closet,sala,comedor,bano_privado,bano_social,bano_servicio,cocina,estudio,balcon,terraza,patio_interior,jardin,zona_de_ropas,zona_verde_privada,local,oficina,bodega,observaciones_dependencias,estado_acabados_pisos,calidad_acabados_pisos,estado_acabados_muros,calidad_acabados_muros,estado_acabados_techos,calidad_acabados_techos,estado_acabados_madera,calidad_acabados_madera,estado_acabados_metal,calidad_acabados_metal,estado_acabados_banos,calidad_acabados_banos,estado_acabados_cocina,calidad_acabados_cocina,tipo_garaje,numero_total_de_garajes,total_cupos_parquedaro,numero_garaje_1,matricula_garaje_1,garaje_cubierto_1,garaje_doble_1,garaje_paralelo_1,garaje_servidumbre_1,numero_garaje_2,matricula_garaje_2,garaje_cubierto_2,garaje_doble_2,garaje_paralelo_2,garaje_servidumbre_2,numero_garaje_3,matricula_garaje_3,garaje_cubierto_3,garaje_doble_3,garaje_paralelo_3,garaje_servidumbre_3,numero_garaje_4,matricula_garaje_4,garaje_cubierto_4,garaje_doble_4,garaje_paralelo_4,garaje_servidumbre_4,numero_garaje_5,matricula_garaje_5,garaje_cubierto_5,garaje_doble_5,garaje_paralelo_5,garaje_servidumbre_5,tipo_deposito,numero_total_depositos,numero_deposito_1,matricula_inmobiliaria_deposito_1,numero_deposito_2,matricula_inmobiliaria_deposito_2,numero_deposito_3,matricula_inmobiliaria_deposito_3,numero_deposito_4,matricula_inmobiliaria_deposito_4,numero_deposito_5,matricula_inmobiliaria_deposito_5,observaciones_generales_construccion,metodo_valuacion_1,concepto_del_metodo_1,metodo_valuacion_2,concepto_del_metodo_2,metodo_valuacion_3,concepto_del_metodo_3,metodo_valuacion_4,concepto_del_metodo_4,metodo_valuacion_5,concepto_del_metodo_5,metodo_valuacion_6,concepto_del_metodo_6,metodo_valuacion_7,concepto_del_metodo_7,metodo_valuacion_8,concepto_del_metodo_8,metodo_valuacion_9,concepto_del_metodo_9,area_privada,valor_area_privada,area_garaje,valor_area_garaje,area_deposito,valor_area_deposito,area_terreno,valor_area_terreno,area_construccion,valor_area_construccion,area_otros,valor_area_otros,area_libre,valor_area_libre,valor_total_avaluo,valor_uvr,valor_avaluo_en_uvr,Longitud,Latitud,id,clean_valor_total_avaluo,clean_valor_uvr,clean_valor_

### 4. Corrección de codificación de caracteres

Se identifican problemas de codificación de caracteres para algunos campos en algunos registros, se logra concluir que estos errores se deben a una codificación incorrecta de formato latino a utf-8. A continuación, se realiza la correción.

In [43]:
for c in df_train_clean.columns:
    if df_train_clean[c].dtype == "O":
        df_train_clean[c] = df_train_clean[c].apply(lambda v: v.encode('latin1','ignore').decode("utf-8",'ignore') if type(v) is str and 'Ã' in v else v)

In [44]:
df_train_clean.head()

,fecha_aprobación,objeto,motivo,proposito,tipo_avaluo,tipo_credito,tipo_subsidio,departamento_inmueble,municipio_inmueble,barrio,sector,direccion_inmueble_informe,alcantarillado_en_el_sector,acueducto_en_el_sector,gas_en_el_sector,energia_en_el_sector,telefono_en_el_sector,vias_pavimentadas,sardineles_en_las_vias,andenes_en_las_vias,estrato,barrio_legal,topografia_sector,condiciones_salubridad,transporte,demanda_interes,paradero,alumbrado,arborizacion,alamedas,ciclo_rutas,nivel_equipamiento_comercial,descripcion_general_sector,perspectivas_de_valorizacion,actualidad_edificadora,comportamiento_oferta_demanda,alcantarillado_en_el_predio,acueducto_en_el_predio,gas_en_el_predio,energia_en_el_predio,telefono_en_el_predio,tipo_inmueble,descripcion_tipo_inmueble,uso_actual,descripcion_uso_inmueble,clase_inmueble,descripcion_clase_inmueble,ocupante,sometido_a_propiedad_horizontal,altura_permitida,observaciones_altura_permitida,aislamiento_posterior,observaciones_aislamiento_posterior,aislamiento_lateral,observaciones_aislamiento_lateral,antejardin,observaciones_antejardin,indice_ocupacion,observaciones_indice_ocupacion,indice_construccion,observaciones_indice_construccion,predio_subdividido_fisicamente,unidades,contadores_agua,contadores_luz,accesorios,area_valorada,observaciones_generales_inmueble,condicion_ph,numero_piso,numero_de_edificios,area_actividad,uso_principal_ph,rph,porteria,citofono,bicicletero,piscina,tanque_de_agua,club_house,garaje_visitantes,teatrino,sauna,vigilancia_privada,tipo_vigilancia,administracion,vetustez,pisos_bodega,estructura,ajustes_sismoresistentes,cubierta,fachada,tipo_fachada,estructura_reforzada,danos_previos,material_de_construccion,detalle_material,iluminacion,ventilacion,irregularidad_planta,irregularidad_altura,observaciones_estructura,habitaciones,estar_habitacion,cuarto_servicio,closet,sala,comedor,bano_privado,bano_social,bano_servicio,cocina,estudio,balcon,terraza,patio_interior,jardin,zona_de_ropas,zona_verde_privada,local,oficina,bodega,observaciones_dependencias,estado_acabados_pisos,calidad_acabados_pisos,estado_acabados_muros,calidad_acabados_muros,estado_acabados_techos,calidad_acabados_techos,estado_acabados_madera,calidad_acabados_madera,estado_acabados_metal,calidad_acabados_metal,estado_acabados_banos,calidad_acabados_banos,estado_acabados_cocina,calidad_acabados_cocina,tipo_garaje,numero_total_de_garajes,total_cupos_parquedaro,numero_garaje_1,matricula_garaje_1,garaje_cubierto_1,garaje_doble_1,garaje_paralelo_1,garaje_servidumbre_1,numero_garaje_2,matricula_garaje_2,garaje_cubierto_2,garaje_doble_2,garaje_paralelo_2,garaje_servidumbre_2,numero_garaje_3,matricula_garaje_3,garaje_cubierto_3,garaje_doble_3,garaje_paralelo_3,garaje_servidumbre_3,numero_garaje_4,matricula_garaje_4,garaje_cubierto_4,garaje_doble_4,garaje_paralelo_4,garaje_servidumbre_4,numero_garaje_5,matricula_garaje_5,garaje_cubierto_5,garaje_doble_5,garaje_paralelo_5,garaje_servidumbre_5,tipo_deposito,numero_total_depositos,numero_deposito_1,matricula_inmobiliaria_deposito_1,numero_deposito_2,matricula_inmobiliaria_deposito_2,numero_deposito_3,matricula_inmobiliaria_deposito_3,numero_deposito_4,matricula_inmobiliaria_deposito_4,numero_deposito_5,matricula_inmobiliaria_deposito_5,observaciones_generales_construccion,metodo_valuacion_1,concepto_del_metodo_1,metodo_valuacion_2,concepto_del_metodo_2,metodo_valuacion_3,concepto_del_metodo_3,metodo_valuacion_4,concepto_del_metodo_4,metodo_valuacion_5,concepto_del_metodo_5,metodo_valuacion_6,concepto_del_metodo_6,metodo_valuacion_7,concepto_del_metodo_7,metodo_valuacion_8,concepto_del_metodo_8,metodo_valuacion_9,concepto_del_metodo_9,area_privada,valor_area_privada,area_garaje,valor_area_garaje,area_deposito,valor_area_deposito,area_terreno,valor_area_terreno,area_construccion,valor_area_construccion,area_otros,valor_area_otros,area_libre,valor_area_libre,valor_total_avaluo,valor_uvr,valor_avaluo_en_uvr,Longitud,Latitud,id,clean_valor_total_avaluo,clean_valor_uvr,clean_valor_

### 5. Tratamiento de campos de área

In [45]:
campos_areas = ['area_privada', 'area_garaje', 'area_deposito', 'area_terreno',
                'area_construccion', 'area_valorada']

for c in campos_areas:
    if df_train_clean[c].dtype == "O":
        df_train_clean[c] = df_train_clean[c].apply(lambda v: float(v.replace(",",".")) if type(v) is str and isfloat(v.replace(",",".")) else np.nan)

In [46]:
df_train_clean['area_total'] = df_train_clean['area_privada'] + df_train_clean['area_garaje'] + df_train_clean['area_deposito'] + df_train_clean['area_terreno'] + df_train_clean['area_construccion']

C:\Users\Santi\AppData\Local\Temp\ipykernel_25592\1736115494.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train_clean['area_total'] = df_train_clean['area_privada'] + df_train_clean['area_garaje'] + df_train_clean['area_deposito'] + df_train_clean['area_terreno'] + df_train_clean['area_construccion']


In [47]:
df_train_clean['diff_area'] = abs(df_train_clean['area_total'] - df_train_clean['area_valorada'])

C:\Users\Santi\AppData\Local\Temp\ipykernel_25592\3536661324.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train_clean['diff_area'] = abs(df_train_clean['area_total'] - df_train_clean['area_valorada'])


In [48]:
df_train_clean[['area_total', 'area_valorada', 'diff_area', 'clean_valor_total_avaluo']].sort_values("diff_area", ascending=False).query("diff_area>1000")

,area_total,area_valorada,diff_area,clean_valor_total_avaluo
2596,3118581.000000000,887.000000000,3117694.000000000,3530624600.162940025
7872,2885763.000000000,0.000000000,2885763.000000000,2590829687.951294899
2583,1600926.000000000,126.000000000,1600800.000000000,638100000.000000000
3474,1150392.000000000,126.000000000,1150266.000000000,469149750.000000000
2341,896.580000000,376582.000000000,375685.420000000,1234180500.000000000
658,755.030000000,310412.000000000,309656.970000000,920403580.000000000
9213,300226.400000000,98.400000000,300128.000000000,556200000.000000000
2814,240125.740000000,86.100000000,240039.640000000,308991400.000000000
11951,367.730000000,235760.000000000,235392.270000000,484095000.000000000
3331,229.280000000,229278.000000000,229048.720000000,346212800.000000000


In [49]:
df_train_clean['area_valorada'] = df_train_clean.apply(lambda r: min(r['area_valorada'],r['area_total']) if r['area_valorada'] > 0 and r['area_total'] > 0 else r['area_valorada'], axis=1)

In [50]:
df_train_clean.query("area_valorada==0").shape

(81, 228)

In [51]:
df_train_clean['area_valorada'] = df_train_clean.apply(lambda r: r['area_total'] if r['area_valorada'] == 0 else r['area_valorada'], axis=1)

In [52]:
df_train_clean.to_excel("../datos/clean_entrenamiento_precios_vivienda.xlsx", index=False)